In [ ]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from utils.utilities import move_data_to_device

from interpretability_utilities import plot_layer_attribution_importance, plot_frame_attributions, load_workspace_file

import pandas as pd

import numpy as np
import librosa
import librosa.display
from scipy.io import wavfile
from scipy.stats import norm

import mlflow
import torch

import matplotlib.pyplot as plt
import seaborn as sns

from captum.attr import DeepLift, NeuronDeepLift, LayerDeepLift
from captum.concept import Concept, TCAV

from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset
from captum.concept._utils.common import concepts_to_str

## Settings and Utils

In [ ]:
# Adjust according to your experiment
ref_fold = "8"
run_id = ""
tracking_server = ""
workspace_file = r""
dataset_dir = r""
device = 'cuda' if torch.cuda.is_available() else 'cpu'

mlflow.set_tracking_uri(f"{tracking_server}:5000")
logged_model = mlflow.pytorch.load_model(f"runs:/{run_id}/models")
logged_model = logged_model.eval()

client = mlflow.MlflowClient()
run = client.get_run(run_id)
run_data = run.data
tags = run_data.tags
sr = int(tags["sample_rate"])
n_fft = int(tags["window_size"])
hop_length = int(tags["hop_size"])
window_size = int(tags["window_size"])

In [ ]:
inp_data, indexes, labels, _, lb_to_idx = load_workspace_file(workspace_file, ref_fold,
                                dataset_dir, device)

idx_to_label = {idx: label for label, idx in lb_to_idx.items()}
target = [idx for label, idx in lb_to_idx.items() if label.startswith("albilora")]

inp_data.requires_grad_()

In [ ]:
rng = np.random.default_rng(135)

Until this execution, only 'others' class was predicted, I'll take two random audios and define concepts from each one for TCAV

In [ ]:
np_labels = np.array(labels)
others_audios = np.where(np.argmax(np_labels, axis=1) == lb_to_idx["others"])[0]

audios_ref_indices = rng.choice(others_audios, replace=False, size=2)
ref_audios = inp_data[audios_ref_indices]

In [ ]:
def filter_audio_frequencies(audio, fmin, fmax, sr, n_fft, hop_size):
    stft_mag, stft_phase = librosa.magphase(
        librosa.stft(audio,
            n_fft=n_fft, win_length=window_size, hop_length=hop_size, center=True)
    )

    frequencies = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

    # filter undesired frequencies
    less_than_min = frequencies < fmin
    greather_than_max = frequencies > fmax

    # Join the fmin and fmax undesired frequencies
    undesired_frequencies = np.flatnonzero(np.logical_or(less_than_min, greather_than_max))

    new_mag = stft_mag

    for i in undesired_frequencies:
        new_mag[i] = 0.0

    filtered_audio = librosa.istft(new_mag * stft_phase, hop_length=hop_size,
                                    win_length=window_size, window="hann", center=True,
                                    length=len(audio))

    return filtered_audio

References to use

- 2kHz-4kHz frequency range of an audio
- 4kHz-6kHz frequency range of an audio
- 6kHz-8kHz frequency range of an audio
- 8kHz-10kHz frequency range of an audio
- Gaussian noise
- Impulsive noise

## Shrikumar et al. (2017) - DeepLift - Rescale Rule

In [ ]:
def set_reference(method, audio, **kwargs):
    """Modifies an audio for compare references"""
    new_audio = audio.copy()
    if method == "gaussian":
        kwargs.setdefault('cycles', 1)
        for i in range(kwargs["cycles"]):
            new_audio += rng.normal(size=len(new_audio)) - rng.normal(size=len(new_audio))

    elif method == "impulsive":
        noise = rng.normal(size=len(new_audio))
        new_audio = np.where(noise > 0.85, new_audio + noise, new_audio)

    else:
        freqs = method.split('-')
        assert len(freqs) == 2, "Invalid values for min frequency and max frequency"
        fmin, fmax = float(freqs[0])*1000, float(freqs[1])*1000

        new_audio = filter_audio_frequencies(new_audio, fmin, fmax, kwargs["sr"],
                                            kwargs["n_fft"], kwargs["hop_size"])

    return new_audio

In [ ]:
baseline_2k = []
for audio in inp_data:
    baseline_2k.append(set_reference("2-4", audio.cpu().detach().numpy(), **{"sr": sr, "n_fft": n_fft, "hop_size": hop_length}))

baseline_2k = move_data_to_device(np.array(baseline_2k), device)

In [ ]:
baseline_4k = []
for audio in inp_data:
    baseline_4k.append(set_reference("4-6", audio.cpu().detach().numpy(), **{"sr": sr, "n_fft": n_fft, "hop_size": hop_length}))

baseline_4k = move_data_to_device(np.array(baseline_4k), device)

In [ ]:
baseline_6k = []
for audio in inp_data:
    baseline_6k.append(set_reference("6-8", audio.cpu().detach().numpy(), **{"sr": sr, "n_fft": n_fft, "hop_size": hop_length}))

baseline_6k = move_data_to_device(np.array(baseline_6k), device)

In [ ]:
baseline_8k = []
for audio in inp_data:
    baseline_8k.append(set_reference("8-10", audio.cpu().detach().numpy(), **{"sr": sr, "n_fft": n_fft, "hop_size": hop_length}))

baseline_8k = move_data_to_device(np.array(baseline_8k), device)

In [ ]:
baseline_gaussian = []
for audio in inp_data:
    baseline_gaussian.append(set_reference("gaussian", audio.cpu().detach().numpy(), **{"sr": sr, "n_fft": n_fft, "hop_size": hop_length}))

baseline_gaussian = move_data_to_device(np.array(baseline_gaussian), device)

In [ ]:
baseline_impulsive = []
for audio in inp_data:
    baseline_impulsive.append(set_reference("impulsive", audio.cpu().detach().numpy(), **{"sr": sr, "n_fft": n_fft, "hop_size": hop_length}))

baseline_impulsive = move_data_to_device(np.array(baseline_impulsive), device)

### Model attribution

In [ ]:
dl = DeepLift(logged_model)

In [ ]:
def model_importance(dl, inp_data, baseline, target, label):
    """Computes the frames importance given a reference and the target classes"""

    attributions = dl.attribute(inp_data, baseline, target=target)
    plot_frame_attributions(attributions, title=f"Average frames importance for {label} using DeepLift")
    return attributions

#### Silent audio

0 tensor

Compute attributions for S. albilora call and syllables

In [ ]:
ca_attribution = dl.attribute(inp_data, target=target[0])

In [ ]:
syl1_attribution = dl.attribute(inp_data, target=target[1])

In [ ]:
syl2_attribution = dl.attribute(inp_data, target=target[2])

In [ ]:
plot_frame_attributions(ca_attribution, title="Average Frames importance using DeepLift")

In [ ]:
plot_frame_attributions(syl1_attribution, "Average Frames importance using DeepLift")

In [ ]:
plot_frame_attributions(syl2_attribution, "Average Frames importance using DeepLift")

In [ ]:
# The detected class(es) for the first experiment with 100 iterations
others_attributions = deconv.attribute(inp_data, target=lb_to_idx["others"])

In [ ]:
plot_frame_attributions(others_attributions, "Average Frames importance using DeepLift")

Behavior for background

In [ ]:
savanna = [idx for label, idx in lb_to_idx.items() if label.startswith("SAV")]

In [ ]:
sav_dry_attributions = deconv.attribute(inp_data, target=savanna[0])
plot_frame_attributions(sav_dry_attributions, "Average Frames importance using DeepLift")

In [ ]:
sav_wet_attributions = deconv.attribute(inp_data, target=savanna[1])
plot_frame_attributions(sav_wet_attributions, "Average Frames importance using DeepLift")

In [ ]:
forest = [idx for label, idx in lb_to_idx.items() if label.startswith("FOR")]

In [ ]:
for_dry_attributions = deconv.attribute(inp_data, target=forest[0])
plot_frame_attributions(for_dry_attributions, "Average Frames importance using DeepLift")

In [ ]:
for_wet_attributions = deconv.attribute(inp_data, target=forest[1])
plot_frame_attributions(for_wet_attributions, "Average Frames importance using DeepLift")

#### Frequency range and noise

2kHz-4kHz activity

In [ ]:
ca_attributions = model_importance(dl, inp_data, baseline_2k, target[0], "Synallaxis Albilora call")
syl1_attributions = model_importance(dl, inp_data, baseline_2k, target[1], "Synallaxis Albilora syllable 1")
syl2_attributions = model_importance(dl, inp_data, baseline_2k, target[2], "Synallaxis Albilora syllable 2")

others_attributions = model_importance(dl, inp_data, baseline_2k, lb_to_idx["others"], "Other birds call")

4kHz-6kHz activity

In [ ]:
ca_attributions = model_importance(dl, inp_data, baseline_4k, target[0], "Synallaxis Albilora call")
syl1_attributions = model_importance(dl, inp_data, baseline_4k, target[1], "Synallaxis Albilora syllable 1")
syl2_attributions = model_importance(dl, inp_data, baseline_4k, target[2], "Synallaxis Albilora syllable 2")

others_attributions = model_importance(dl, inp_data, baseline_4k, lb_to_idx["others"], "Other birds call")

6kHz-8kHz activity

In [ ]:
ca_attributions = model_importance(dl, inp_data, baseline_6k, target[0], "Synallaxis Albilora call")
syl1_attributions = model_importance(dl, inp_data, baseline_6k, target[1], "Synallaxis Albilora syllable 1")
syl2_attributions = model_importance(dl, inp_data, baseline_6k, target[2], "Synallaxis Albilora syllable 2")

others_attributions = model_importance(dl, inp_data, baseline_6k, lb_to_idx["others"], "Other birds call")

8kHz-10kHz activity

In [ ]:
ca_attributions = model_importance(dl, inp_data, baseline_8k, target[0], "Synallaxis Albilora call")
syl1_attributions = model_importance(dl, inp_data, baseline_8k, target[1], "Synallaxis Albilora syllable 1")
syl2_attributions = model_importance(dl, inp_data, baseline_8k, target[2], "Synallaxis Albilora syllable 2")

others_attributions = model_importance(dl, inp_data, baseline_8k, lb_to_idx["others"], "Other birds call")

Gaussian noise

In [ ]:
ca_attributions = model_importance(dl, inp_data, baseline_gaussian, target[0], "Synallaxis Albilora call")
syl1_attributions = model_importance(dl, inp_data, baseline_gaussian, target[1], "Synallaxis Albilora syllable 1")
syl2_attributions = model_importance(dl, inp_data, baseline_gaussian, target[2], "Synallaxis Albilora syllable 2")

others_attributions = model_importance(dl, inp_data, baseline_gaussian, lb_to_idx["others"], "Other birds call")

Impulsive noise

In [ ]:
ca_attributions = model_importance(dl, inp_data, baseline_impulsive, target[0], "Synallaxis Albilora call")
syl1_attributions = model_importance(dl, inp_data, baseline_impulsive, target[1], "Synallaxis Albilora syllable 1")
syl2_attributions = model_importance(dl, inp_data, baseline_impulsive, target[2], "Synallaxis Albilora syllable 2")

others_attributions = model_importance(dl, inp_data, baseline_impulsive, lb_to_idx["others"], "Other birds call")

### Block attribution

#### First block

In [ ]:
dl_first_conv_block = LayerDeepLift(logged_model, logged_model.base.conv_block1)

##### Silent audio

In [ ]:
ca_attributions = dl_first_conv_block.attribute(inp_data, target=target[0])

In [ ]:
syl1_attributions = dl_first_conv_block.attribute(inp_data, target=target[1])

In [ ]:
syl2_attributions = dl_first_conv_block.attribute(inp_data, target=target[2])

In [ ]:
others_attributions = dl_first_conv_block.attribute(inp_data, target=lb_to_idx["others"])

In [ ]:
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

##### Frequency range and noise

2kHz-4kHz activity

In [ ]:
ca_attributions = dl_first_conv_block.attribute(inp_data, baseline_2k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_first_conv_block.attribute(inp_data, baseline_2k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_first_conv_block.attribute(inp_data, baseline_2k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_first_conv_block.attribute(inp_data, baseline_2k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

4kHz-6kHz activity

In [ ]:
ca_attributions = dl_first_conv_block.attribute(inp_data, baseline_4k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_first_conv_block.attribute(inp_data, baseline_4k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_first_conv_block.attribute(inp_data, baseline_4k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_first_conv_block.attribute(inp_data, baseline_4k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

6kHz-8kHz activity

In [ ]:
ca_attributions = dl_first_conv_block.attribute(inp_data, baseline_6k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_first_conv_block.attribute(inp_data, baseline_6k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_first_conv_block.attribute(inp_data, baseline_6k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_first_conv_block.attribute(inp_data, baseline_6k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

8kHz-10kHz activity

In [ ]:
ca_attributions = dl_first_conv_block.attribute(inp_data, baseline_8k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_first_conv_block.attribute(inp_data, baseline_8k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_first_conv_block.attribute(inp_data, baseline_8k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_first_conv_block.attribute(inp_data, baseline_8k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

Gaussian noise

In [ ]:
ca_attributions = dl_first_conv_block.attribute(inp_data, baseline_gaussian, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_first_conv_block.attribute(inp_data, baseline_gaussian, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_first_conv_block.attribute(inp_data, baseline_gaussian, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_first_conv_block.attribute(inp_data, baseline_gaussian, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

Impulsive noise

In [ ]:
ca_attributions = dl_first_conv_block.attribute(inp_data, baseline_impulsive, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_first_conv_block.attribute(inp_data, baseline_impulsive, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_first_conv_block.attribute(inp_data, baseline_impulsive, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_first_conv_block.attribute(inp_data, baseline_impulsive, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

#### 2th block

In [ ]:
dl_second_conv_block = LayerDeepLift(logged_model, logged_model.base.conv_block2)

##### Silent audio

In [ ]:
ca_attributions = dl_second_conv_block.attribute(inp_data, target=target[0])

In [ ]:
syl1_attributions = dl_second_conv_block.attribute(inp_data, target=target[1])

In [ ]:
syl2_attributions = dl_second_conv_block.attribute(inp_data, target=target[2])

In [ ]:
others_attributions = dl_second_conv_block.attribute(inp_data, target=lb_to_idx["others"])

In [ ]:
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

##### Frequency range and noise

2kHz-4kHz activity

In [ ]:
ca_attributions = dl_second_conv_block.attribute(inp_data, baseline_2k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_second_conv_block.attribute(inp_data, baseline_2k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_second_conv_block.attribute(inp_data, baseline_2k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_second_conv_block.attribute(inp_data, baseline_2k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

4kHz-6kHz activity

In [ ]:
ca_attributions = dl_second_conv_block.attribute(inp_data, baseline_4k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_second_conv_block.attribute(inp_data, baseline_4k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_second_conv_block.attribute(inp_data, baseline_4k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_second_conv_block.attribute(inp_data, baseline_4k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

6kHz-8kHz activity

In [ ]:
ca_attributions = dl_second_conv_block.attribute(inp_data, baseline_6k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_second_conv_block.attribute(inp_data, baseline_6k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_second_conv_block.attribute(inp_data, baseline_6k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_second_conv_block.attribute(inp_data, baseline_6k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

8kHz-10kHz activity

In [ ]:
ca_attributions = dl_second_conv_block.attribute(inp_data, baseline_8k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_second_conv_block.attribute(inp_data, baseline_8k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_second_conv_block.attribute(inp_data, baseline_8k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_second_conv_block.attribute(inp_data, baseline_8k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

Gaussian noise

In [ ]:
ca_attributions = dl_second_conv_block.attribute(inp_data, baseline_gaussian, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_second_conv_block.attribute(inp_data, baseline_gaussian, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_second_conv_block.attribute(inp_data, baseline_gaussian, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_second_conv_block.attribute(inp_data, baseline_gaussian, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

Impulsive noise

In [ ]:
ca_attributions = dl_second_conv_block.attribute(inp_data, baseline_impulsive, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_second_conv_block.attribute(inp_data, baseline_impulsive, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_second_conv_block.attribute(inp_data, baseline_impulsive, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_second_conv_block.attribute(inp_data, baseline_impulsive, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

#### 5th block

In [ ]:
dl_fifth_conv_block = LayerDeepLift(logged_model, logged_model.base.conv_block5)

##### Silent audio

In [ ]:
ca_attributions = dl_fifth_conv_block.attribute(inp_data, target=target[0])

In [ ]:
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_fifth_conv_block.attribute(inp_data, target=target[1])

In [ ]:
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_fifth_conv_block.attribute(inp_data, target=target[2])

In [ ]:
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_fifth_conv_block.attribute(inp_data, target=lb_to_idx["others"])

In [ ]:
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

##### Frequency range and noise

2kHz-4kHz activity

In [ ]:
ca_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_2k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_2k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_2k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_2k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

4kHz-6kHz activity

In [ ]:
ca_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_4k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_4k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_4k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_4k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

6kHz-8kHz activity

In [ ]:
ca_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_6k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_6k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_6k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_6k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

8kHz-10kHz activity

In [ ]:
ca_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_8k, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_8k, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_8k, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_8k, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

Gaussian noise

In [ ]:
ca_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_gaussian, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_gaussian, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_gaussian, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_gaussian, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

Impulsive noise

In [ ]:
ca_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_impulsive, target[0])
plot_layer_attribution_importance(ca_attributions, "Average importance for Synallaxis Albilora call using DeepLift")

In [ ]:
syl1_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_impulsive, target[1])
plot_layer_attribution_importance(syl1_attributions, "Average importance for Synallaxis Albilora 1th syllable using DeepLift")

In [ ]:
syl2_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_impulsive, target[2])
plot_layer_attribution_importance(syl2_attributions, "Average importance for Synallaxis Albilora 2th syllable using DeepLift")

In [ ]:
others_attributions = dl_fifth_conv_block.attribute(inp_data, baseline_impulsive, lb_to_idx["others"])
plot_layer_attribution_importance(others_attributions, "Average importance for other birds call using DeepLift")

## Kim et al. (2018) - TCAV